# Drivers of shoreline evolution

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_10896\2707390607.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("07_Drivers")
dataset_dir_driver = dataset_dir.joinpath("drivers.nc")
dataset_out_file = "drivers"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [4]:
# # # # write csv to netcdf

# # # # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\11202268-hydraulic-engineering\MSc_students\Dante_van_der_Heijden\02. Data\01. Final_data\csv_files\all_drivers.csv'
# df = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir_driver)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [5]:
# open datasets
df_driver = xr.open_dataset(dataset_dir_driver)

# check original dataset
df_driver

<xarray.Dataset>
Dimensions:            (index: 57924)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 57919 57920 57921 57922 57923
Data variables: (12/22)
    transect_id        (index) object ...
    country_id         (index) object ...
    continent          (index) object ...
    country_name       (index) object ...
    changerate         (index) float64 ...
    transect_origin_x  (index) float64 ...
    ...                 ...
    Tfixed             (index) float64 ...
    nourish_label      (index) object ...
    nourish_date       (index) object ...
    hotspot_id         (index) object ...
    ldb_label          (index) object ...
    Type               (index) object ...

<IPython.core.display.Javascript object>

In [6]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_driver, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [7]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_driver, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\07_Drivers\\drivers.nc': {'warnings': '37', 'errors': '12'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [8]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\07_Drivers\metadata_shorelinemonitor_drivers.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    df_driver.attrs[attr_name] = attr_val

df_driver.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [9]:
df_driver

<xarray.Dataset>
Dimensions:            (index: 57924)
Coordinates:
  * index              (index) int64 0 1 2 3 4 ... 57919 57920 57921 57922 57923
Data variables: (12/22)
    transect_id        (index) object ...
    country_id         (index) object ...
    continent          (index) object ...
    country_name       (index) object ...
    changerate         (index) float64 ...
    transect_origin_x  (index) float64 ...
    ...                 ...
    Tfixed             (index) float64 ...
    nourish_label      (index) object ...
    nourish_date       (index) object ...
    hotspot_id         (index) object ...
    ldb_label          (index) object ...
    Type               (index) object ...
Attributes: (12/21)
    TITLE:               The Shoreline Monitor Drivers
    TITLE_ABBREVIATION:  shore_mon_drivers
    DESCRIPTION:         The Shoreline Monitor Drivers dataset aims to identi...
    SHORT_DESCRIPTION:   The Shoreline Monitor Drivers dataset aims to identi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_DRIVERS
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [10]:
start_lats = []
for x in df_driver['transect_origin_y'].values:
    try:
        x2 = float(x)
    except:
        x2 = float(str(x)[:-2])
    finally:
        start_lats.append(x2)

<IPython.core.display.Javascript object>

In [11]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = df_driver["transect_origin_x"].values
end_lons = df_driver["transect_end_x"].values
end_lats = df_driver["transect_end_y"].values
coords = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

df_driver["transect_geom"] = (
    ["index"],
    [str(LineString(line)) for line in coords],
)
df_driver["transect_geom"].attrs["long_name"] = "Transect Geometry"

<IPython.core.display.Javascript object>

In [12]:
df_driver = df_driver.drop('transect_origin_y')
df_driver["transect_origin_y"] = ("index", start_lats,)

<IPython.core.display.Javascript object>

In [13]:
df_driver = df_driver.drop(['hotspot_id', 'transect_end_x', 'transect_end_y', 'P_mis'])

<IPython.core.display.Javascript object>

In [14]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
df_driver = df_driver.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\07_Drivers\vars_drivers.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    df_driver = df_driver.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            df_driver[var_dict['name']].attrs[key] = value

<IPython.core.display.Javascript object>

In [15]:
# change dtypes
object_vars = ['transect_id', 'country_id', 'country', 'continent', 'transect_geom', 'hotspot_id', 
               'seasonal_id_conf', 'reclamation_id_conf', 'nourishment_id_conf', 'littoraldb_id_conf', 'date_nourishment', 'ldb_type']
for ov in object_vars:
    df_driver[ov] = df_driver[ov].astype('S')

<IPython.core.display.Javascript object>

In [16]:
# change dtypes
object_vars = ['lon', 'lat', 'changerate', 'seasonal_displacement', 't_min_seasonal_sp', 't_max_seasonal_sp', 't_recl_construction']
for ov in object_vars:
    df_driver[ov] = df_driver[ov].astype('float')

<IPython.core.display.Javascript object>

In [17]:
all_vars = list(df_driver.keys())
data_vars = ['seasonal_displacement', 't_min_seasonal_sp', 't_max_seasonal_sp', 
             't_recl_construction', 'date_nourishment', 'label_littoraldb', 'ldb_type', 'changerate',
             'seasonal_id_conf', 'reclamation_id_conf', 'nourishment_id_conf', 'littoraldb_id_conf',]
# set some data variables to coordinates to avoid duplication of dimensions in later stage
df_driver = df_driver.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
df_driver = df_driver.drop('index')

<IPython.core.display.Javascript object>

In [18]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
df_driver

<xarray.Dataset>
Dimensions:                (stations: 57924)
Coordinates:
    transect_id            (stations) |S15 b'BOX_145_168_39' ... b'BOX_146_05...
    country_id             (stations) |S7 b'KWT' b'PRT' b'MEX' ... b'SAU' b'ARE'
    continent              (stations) |S13 b'Asia' b'Europe' ... b'Asia' b'Asia'
    country                (stations) |S27 b'Kuwait' ... b'United Arab Emirates'
    lon                    (stations) float64 47.89 -8.654 -109.4 ... 49.12 54.4
    hotspot_id             (stations) |S31 b'prograding_feature_polygon_928' ...
    transect_geom          (stations) |S57 b'LINESTRING (47.885022 29.319149,...
    lat                    (stations) float64 29.32 40.92 26.11 ... 27.42 24.4
Dimensions without coordinates: stations
Data variables:
    changerate             (stations) float64 3.437 -7.711 ... 2.404 26.83
    seasonal_id_conf       (stations) |S7 b'low' b'high' ... b'low' b'low'
    seasonal_displacement  (stations) float64 423.0 21.0 204.0 ... 717.0 3.0
    t_min_seasonal_sp      (stations) float64 9.0 7.0 10.0 11.0 ... 2.0 9.0 4.0
    t_max_seasonal_sp      (stations) float64 4.0 12.0 5.0 4.0 ... 8.0 3.0 11.0
    reclamation_id_conf    (stations) |S6 b'low' b'low' b'low' ... b'low' b'low'
    t_recl_construction    (stations) float64 -999.0 -999.0 ... 2.011e+03
    nourishment_id_conf    (stations) |S6 b'low' b'low' b'low' ... b'low' b'low'
    date_nourishment       (stations) |S30 b'[]' b'[]' b'[]' ... b'[]' b'[]'
    littoraldb_id_conf     (stations) |S6 b'low' b'low' b'low' ... b'low' b'low'
    ldb_type               (stations) |S14 b'Splitted' ... b'Updrift'
Attributes: (12/21)
    TITLE:               The Shoreline Monitor Drivers
    TITLE_ABBREVIATION:  shore_mon_drivers
    DESCRIPTION:         The Shoreline Monitor Drivers dataset aims to identi...
    SHORT_DESCRIPTION:   The Shoreline Monitor Drivers dataset aims to identi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 
    LONG_NAME:           THE_SHORELINE_MONITOR_DRIVERS
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [19]:
# save new .nc files
df_driver.to_netcdf(path=str(dataset_dir_driver).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [20]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_driver).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [21]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_driver).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\07_Drivers\\drivers_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [22]:
# export to zarr in write mode (to overwrite if exists)
df_driver.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [23]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>

In [24]:
test

<xarray.Dataset>
Dimensions:                (stations: 57924)
Coordinates:
    continent              (stations) |S13 dask.array<chunksize=(14481,), meta=np.ndarray>
    country                (stations) |S27 dask.array<chunksize=(14481,), meta=np.ndarray>
    country_id             (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    hotspot_id             (stations) |S31 dask.array<chunksize=(14481,), meta=np.ndarray>
    lat                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    lon                    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    transect_geom          (stations) |S57 dask.array<chunksize=(7241,), meta=np.ndarray>
    transect_id            (stations) |S15 dask.array<chunksize=(14481,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate             (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    date_nourishment       (stations) |S30 dask.array<chunksize=(14481,), meta=np.ndarray>
    ldb_type               (stations) |S14 dask.array<chunksize=(14481,), meta=np.ndarray>
    littoraldb_id_conf     (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    nourishment_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    reclamation_id_conf    (stations) |S6 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_displacement  (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    seasonal_id_conf       (stations) |S7 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_max_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_min_seasonal_sp      (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
    t_recl_construction    (stations) float64 dask.array<chunksize=(28962,), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Van der Heijden, D., Luijendijk, A. & Kras, E.
    CITATION:            van der Heijden, D. (2022). Global identification an...
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor Drivers dataset aims to identi...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'shoreline dynamics', 'big dat...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2022-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor Drivers
    TITLE_ABBREVIATION:  shore_mon_drivers
    UNITS:

<IPython.core.display.Javascript object>

In [25]:
np.unique(test['seasonal_id_conf'].values)

array([b'high', b'low', b'medium', b'unknown'], dtype='|S7')

<IPython.core.display.Javascript object>